# Import CSVs into OWID Grapher

The following notebook contains code for importing the UN SDG database into Grapher. It consumes the CSVs generated by a separate notebook.

### Notes

  - The length of the `name` column in the `datasets` table had to be changed to `512`: we have a `UNIQUE` constraint on it and the `namespace` column, and I was getting duplicates: `alter table datasets change name name varchar(512);`
  - We decided that this version of SDG needs to be imported into a new namespace (`un_sdg_2019`), as it was not possible to update the values of the existing version of SDG that is present in Grapher.

In [ ]:
import pandas as pd
from tqdm import tqdm

from db import connection
from db_utils import DBUtils

from import_metadata import extract_description

pd.set_option('display.max_colwidth', -1)

# ID of user who imported the data
USER_ID = 29

# Dataset namespace
NAMESPACE = 'un_sdg_2019'

## Standardizing the entities

We use OpenRefine to standardize the entities. [**Read the instructions in the lc-reconcile repo**](https://github.com/owid/lc-reconcile) to perform the standardization on the `entities.csv` file.

At the end, you should have an `./entities_standardized.csv` file with columns:
- `id` (as the original CSV)
- `name` (as the original CSV)
- `standardized_name` (the database entity name)
- `db_entity_id` (the database entity id)

**There will likely be entities that were not matched with a database entity, those will be inserted below.**

In [ ]:
entities = pd.read_csv('./entities_standardized.csv', index_col='id')

In [ ]:
with connection as c:
    db = DBUtils(c)
    new_entities = entities[entities['db_entity_id'].isnull()]
    for _, entity in new_entities.iterrows():
        entity_id = entity.name
        entity_name = entity['name']
        db_entity_id = db.get_or_create_entity(entity_name)
        entities.loc[entity_id, 'db_entity_id'] = db_entity_id
        print(db_entity_id, entity['name'])

In [ ]:
len(entities[entities['db_entity_id'].isnull()])

## Inserting the data

In [ ]:
# [entity_id] → [db_entity_id] lookup
db_entity_id_by_entity_id = { row.name: int(row['db_entity_id']) for _, row in entities.iterrows() }

In [ ]:
variables = pd.read_csv('./exported_data/variables.csv')
datasets = pd.read_csv('./exported_data/datasets.csv')
sources = pd.read_csv('./exported_data/sources.csv')

In [ ]:
with connection as c:
    db = DBUtils(c)
    
    for _, dataset in tqdm(datasets.iterrows(), total=len(datasets)):
        
        # Insert the dataset
        print("Inserting dataset: %s" % dataset['name'])
        db_dataset_id = db.upsert_dataset(name=dataset['name'], namespace=NAMESPACE, user_id=USER_ID)
        
        # Insert the source
        source = sources[sources['dataset_id'] == dataset.id].iloc[0]
        print("Inserting source: %s" % source['name'])
        db_source_id = db.upsert_source(name=source['name'], description=source['description'], dataset_id=db_dataset_id)
        
        # Insert variables associated with this dataset
        for j, variable in variables[variables.dataset_id == dataset['id']].iterrows():
            # insert row in variables table
            print("Inserting variable: %s" % variable['name'])
            db_variable_id = db.upsert_variable(
                name=variable['name'], 
                code=None, 
                unit=variable['unit'], 
                short_unit=None, 
                source_id=db_source_id, 
                dataset_id=db_dataset_id, 
                description=None)

            # read datapoints
            data_values = pd.read_csv('./exported_data/%04d_datapoints.csv' % variable.id)

            values = [(float(row['value']), int(row['year']), db_entity_id_by_entity_id[row['entity']], db_variable_id)
                      for _, row in data_values.iterrows()]

            print("Inserting values...")
            db.upsert_many("""
                INSERT INTO data_values (value, year, entityId, variableId)
                VALUES (%s, %s, %s, %s)
            """, values)
            print("Inserted %d values for variable" % len(values))


### Useful SQL for deleting all `data_values`, `variables`, `sources`, and `datasets` belonging to a `namespace`

```sql
DELETE data_values
FROM   data_values
       INNER JOIN variables
               ON variables.id = data_values.variableid
       INNER JOIN sources
               ON sources.id = variables.sourceid
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_2019';

DELETE variables
FROM   variables
       INNER JOIN sources
               ON sources.id = variables.sourceid
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_2019';

DELETE sources
FROM   sources
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_2019';

DELETE FROM datasets
WHERE  namespace = 'un_sdg_2019';  
```